# Main Interactive Visual

In [3]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from fuzzywuzzy import process
import unicodedata

# Function to remove accents and special characters
def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return "".join([c for c in nfkd_form if not unicodedata.combining(c)])

transfer_df = pd.read_csv("Transfermarkt_values.csv")
per90_df = pd.read_csv("all_players_per90_stats.csv")
match_rating_df = pd.read_csv("player_match_ratings.csv")

# Convert all different dates to datetime
transfer_df['Date'] = pd.to_datetime(transfer_df['Date'], format='%b %d, %Y')
per90_df['date'] = pd.to_datetime(per90_df['date'], format='%m/%d/%Y')
match_rating_df['Date'] = pd.to_datetime(match_rating_df['Date'], format='%m/%d/%Y %H:%M')

# Standardize player names
transfer_df['Player'] = transfer_df['Player'].str.strip().str.lower()
per90_df['player_name'] = per90_df['player_name'].str.replace('-', ' ').str.strip().str.lower()
match_rating_df['Player'] = match_rating_df['Player'].str.strip().str.lower()

# Removing accents 
transfer_df['Player'] = transfer_df['Player'].apply(remove_accents)
per90_df['player_name'] = per90_df['player_name'].apply(remove_accents)
match_rating_df['Player'] = match_rating_df['Player'].apply(remove_accents)

# match player names using fuzzywuzzy with a quality threshold
def get_full_name(last_name, full_names):
    best_match = process.extractOne(last_name, full_names)
    return best_match[0] if best_match and best_match[1] > 80 else last_name 

# Get all full names from transfer and per90 datasets
full_names = list(set(transfer_df['Player']).union(set(per90_df['player_name'])))
match_rating_df['Player'] = match_rating_df['Player'].apply(lambda x: get_full_name(x, full_names))

# Dropdown options
players = transfer_df['Player'].unique()
per90_metrics = ['goals_per90', 'assists_per90', 'shots_per90', 'sca_per90', 'passes_per90']

# Initial player
initial_player = players[0]

# Get data for the selected player
def get_player_data(player):
    transfer_data = transfer_df[transfer_df['Player'] == player]
    per90_data = per90_df[per90_df['player_name'] == player]
    match_data = match_rating_df[match_rating_df['Player'] == player]
    return transfer_data, per90_data, match_data

# Initialize the figure and traces
transfer_data, per90_data, match_data = get_player_data(initial_player)

# Creating the subplots layout
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, row_heights=[0.2, 0.3, 0.5], vertical_spacing=0.05)

# Transfer Value Line in the Bottom Graph
fig.add_trace(go.Scatter(x=transfer_data['Date'], y=transfer_data['Value (€)'],
                         mode='lines+markers', name='Transfer Value',
                         line=dict(color='blue')),
              row=3, col=1)

# Initialize Per 90 Stats in the middle
fig.add_trace(go.Scatter(x=per90_data['date'], y=per90_data['goals_per90'],
                         mode='lines+markers', name='goals_per90', line=dict(color='green')),
              row=2, col=1)

# colorscale gradient thing from red to orange to green 
custom_colorscale = [
    [0.0, 'rgb(255, 0, 0)'],    # Red at 0
    [0.5, 'rgb(255, 165, 0)'],  # Orange at 5
    [1.0, 'rgb(0, 128, 0)']     # Green at 10
]

# Sofascore Ratings up top 
fig.add_trace(go.Bar(x=match_data['Date'], y=match_data['Match Rating'],
                     marker=dict(
                         color=match_data['Match Rating'],
                         colorscale=custom_colorscale,
                         cmin=0,
                         cmax=10,
                         colorbar=dict(
                             title='Sofascore Rating',
                             titleside='right',
                             ticks='outside',
                             tickvals=[0, 2.5, 5, 7.5, 10],
                             ticktext=['0 (Red)', '2.5', '5 (Orange)', '7.5', '10 (Green)'],
                             len=0.5,
                             thickness=15,  
                             x=1.25,  # Moves the color bar to the right
                             y=0.75,  # Move the color bar to the up and down
                         )
                     ),
                     name='Sofascore Ratings',
                     showlegend=False),  # Disable- don't need now
              row=1, col=1)

# create per90 stat update buttons
def update_per90_trace(stat):
    return {
        'label': stat,
        'method': 'update',
        'args': [
            {'y': [None, per90_data[stat], None]},  # Only update the Per90 trace
            {'title': f'Transfer Value, {stat} & Sofascore Ratings'}
        ],
        'visible': True
    }

# Dropdown buttons for Per 90 Stats- pls only affect middle
buttons = []
for stat in per90_metrics:
    buttons.append({
        'label': stat,
        'method': 'restyle',  
        'args': [{'y': [per90_data[stat]]}, [1]],  
    })

# Players Dropdown which should fully pdates All Graphs
player_buttons = []
for player in players:
    transfer_data, per90_data, match_data = get_player_data(player)
    player_buttons.append({
        'label': player.title(),
        'method': 'update',
        'args': [
            {'x': [transfer_data['Date'], per90_data['date'], match_data['Date']],
             'y': [transfer_data['Value (€)'], per90_data['goals_per90'], match_data['Match Rating']]},
            {'title': f'Transfer Value, Per 90 Stats & Sofascore Ratings - {player.title()}'}
        ]
    })

# Update layout with dropdowns
fig.update_layout(
    updatemenus=[{
        'buttons': buttons,  # Per 90 stat toggle
        'direction': 'down',
        'showactive': True,
        'x': 1.05,  
        'xanchor': 'center',
        'y': 0.6,  
        'yanchor': 'middle'
    },
    {
        'buttons': player_buttons,  # Player switch
        'direction': 'down',
        'showactive': True,
        'x': 1.05,  
        'xanchor': 'center',
        'y': 0.75,  
        'yanchor': 'middle'
    }],
    title=f'Transfer Value, Per 90 Stats & Sofascore Ratings - {initial_player}',
    xaxis_title='Date',
    template='plotly_white',
    height=800
)


## Alternative temp. Graph (Per 90s)

In [6]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

per90_df = pd.read_csv("all_players_per90_stats.csv")
per90_df['date'] = pd.to_datetime(per90_df['date'], format='%m/%d/%Y')
per90_df['player_name'] = per90_df['player_name'].str.replace('-', ' ').str.strip().str.lower()

# Dropdown 
players = per90_df['player_name'].unique()
per90_metrics = ['goals_per90', 'assists_per90', 'shots_per90', 'sca_per90', 'passes_per90']

initial_player = players[0]

# Creating a subplot with 5 rows and 1 column
fig = make_subplots(
    rows=5, 
    cols=1, 
    subplot_titles=per90_metrics,
    vertical_spacing=0.1  # prevent cramp
)

# initial traces for each metric
for i, stat in enumerate(per90_metrics):
    player_data = per90_df[per90_df['player_name'] == initial_player]
    fig.add_trace(
        go.Scatter(
            x=player_data['date'], 
            y=player_data[stat], 
            mode='lines+markers', 
            name=f'{stat} ({initial_player})'
        ),
        row=i+1, col=1
    )

# Player dropdown
player_buttons = []
for player in players:
    player_data = per90_df[per90_df['player_name'] == player]
    button = {
        'label': player.title(),
        'method': 'update',
        'args': [
            {'x': [player_data['date']]*5,
             'y': [player_data[metric] for metric in per90_metrics],
             'name': [f'{metric} ({player})' for metric in per90_metrics]},
            {'title': f'Per 90 Stats for {player}'}
        ]
    }
    player_buttons.append(button)

# Update with dropdown
fig.update_layout(
    updatemenus=[
        {
            'buttons': player_buttons,
            'direction': 'down',
            'showactive': True,
            'x': 1.1,
            'xanchor': 'left',
            'y': 1.1,
            'yanchor': 'top'
        }
    ],
    title=f'Per 90 Stats for {initial_player}',
    template='plotly_white',
    height=1200,  
    margin=dict(l=50, r=50, t=100, b=50)  
)

for i, stat in enumerate(per90_metrics):
    fig.update_yaxes(title_text=stat, row=i+1, col=1)

fig.show()